<h2> Training Deep Neural Networks </h2> 

To tackle complex problems, we need to train much deeper DNN with 10 layers or much more, each containing hundreds of neurons. 

<h3> Issues faced when training DNN </h3> 

- Vanishing gradient/ exploding gradients
- Not enough training data for a large network
- Training will be extemely slow
- A model with millions of parameters would risk overfitting the training set if there is not enough instances. 

<h3> Vanishing/Exploding Gradients Problems </h3> 

- The backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient on the way. 
- Once the gradient for each parameter is calculated, the gradients are used to update each parameter. 
- **However** gradients get smaller and smaller as the algorithm progresses to the lower layers. 

- Due to this vanishing gradient, the weights and biases of the lower layers stay unchanged. 


**The opposite can happen too**

- In some cases, the gradient gets bigger when moving to lower layers, hence the lower layers end up getting large weight updates and the algorithm diverges. 


In a general sense, deep neural networks suffer from unstable gradients throughout the network resulting in different layers learning at different speeds. 

Issues which contribute to this:

- Combination of logistic sigmoid activation function and the weight initialization technique (Random initialization using a normal distribution with a mean of 0 and a standard dev. of 1)

- Looking at the sigmoid activation function, it saturates at large positive and negative inputs and the function saturates at 0 and 1. At these saturated positions, the gradient is close to zero and hence cannot back propagate through a network. Hence there is not enough to be backpropagated to lower layers.
